# Decision tree Classification

## Preperation 

### Importing needed libraries

In [60]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer

### Loading Data

In [69]:
data = pd.read_csv("../../data/220502_train_data_preprocessed.csv", sep=';')
data.head()

,id,label,tweet,n_mentions,hashtags,without_puctioation,tweet_lower,tweet_token,clean_token,clean_hashtags,stemmed_tokens,stemmed_hashtags,lemmatized_tokens,lemmatized_hashtags
0,1,0,@user when a father is dysfunctional and is s...,1,['run'],user when a father is dysfunctional and is so...,user when a father is dysfunctional and is so...,"['when', 'father', 'dysfunctional', 'is', 'sel...","['father', 'dysfunctional', 'selfish', 'drags'...",['run'],"['father', 'dysfunct', 'selfish', 'drag', 'kid...",['run'],"['father', 'dysfunctional', 'selfish', 'drag',...",['run']
1,2,0,@user @user thanks for #lyft credit i can't us...,2,"['lyft', 'disapointed', 'getthanked']",user user thanks for lyft credit i cant use ca...,user user thanks for lyft credit i cant use ca...,"['user', 'for', 'credit', 'cant', 'cause', 'do...","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'caus', 'dont', 'wh...","['lyft', 'disapoint', 'getthank']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","['lyft', 'disapointed', 'getthanked']"
2,3,0,bihday your majesty,0,[],bihday your majesty,bihday your majesty,['your'],[],[],[],[],[],[]
3,4,0,#model i love u take with u all the time in ...,0,['model'],model i love u take with u all the time in u...,model i love u take with u all the time in u...,"['i', 'with', 'u', 'all', 'time', 'urð\x9f\x93...","['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...",['model']
4,5,0,factsguide: society now #motivation,0,['motivation'],factsguide society now motivation,factsguide society now motivation,"['society', 'motivation']","['society', 'motivation']",['motivation'],"['societi', 'motiv']",['motiv'],"['society', 'motivation']",['motivation']


## Try Up-sample Minority Class

Seperate Minortity Class

In [49]:
data_minority = data[data.label == 1]
data_majority = data[data.label == 0]
print("length majority", len(data_majority))
print("length minority", len(data_minority))

length majority 29720
length minority 2242


Upsample minority class

In [50]:
data_minority = resample(data_minority, replace = True, n_samples=29720, random_state=55)

Combine the classes

In [51]:
data_upsampled = pd.concat([data_majority, data_minority])
data_upsampled.label.value_counts()

0    29720
1    29720
Name: label, dtype: int64

In [52]:
data = data_upsampled

## Feature Selection Lemmatized

In [62]:
features = ["n_mentions", "lemmatized_hashtags", "lemmatized_tokens"]
X = data[features]
y = data.label
X.head()

,n_mentions,lemmatized_hashtags,lemmatized_tokens
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',..."
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w..."
2,0,[],[]
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f..."
4,0,['motivation'],"['society', 'motivation']"


## Bag of words model

Create the bag of words from the data

In [63]:
import texthero as hero
from texthero import preprocessing
X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=5000))
X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=5000))
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/3688221143.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_tokens"] = (hero.tfidf(X["lemmatized_tokens"], max_features=5000))
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/3688221143.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_hashtags"] = (hero.tfidf(X["lemmatized_hashtags"], max_features=5000))


,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0,[],[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0,['motivation'],"['society', 'motivation']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
31957,1,[],['user'],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
31958,0,"['shame', 'imwithher']","['see', 'turner', 'trying', 'wrap', 'mantle', ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
31959,0,[],"['song', 'morning']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
31960,1,"['sikh', 'temple', 'calgary', 'wso']","['sikh', 'vandalised', 'wso', 'act']","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Reduce the dimension of the vector

In [64]:
X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2594054688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_tokens"] = (hero.tsne(X["bow_tokens"]))
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2594054688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["bow_hashtags"] = (hero.tsne(X["bow_hashtags"]))


,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[4.522366523742676, -12.791789054870605]","[3.0793397426605225, 12.407670974731445]"
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[-16.068601608276367, -44.1268196105957]","[0.5125786662101746, 1.6410552263259888]"
2,0,[],[],"[-58.391571044921875, 23.286211013793945]","[-0.5365710854530334, -12.591863632202148]"
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[58.901100158691406, 30.188573837280273]","[18.094478607177734, -19.416061401367188]"
4,0,['motivation'],"['society', 'motivation']","[-41.57936477661133, 31.845523834228516]","[7.747095108032227, -33.71648406982422]"
...,...,...,...,...,...
31957,1,[],['user'],"[-20.37633514404297, 61.130393981933594]","[-0.5365710854530334, -12.591863632202148]"
31958,0,"['shame', 'imwithher']","['see', 'turner', 'trying', 'wrap', 'mantle', ...","[3.5070629119873047, 20.457685470581055]","[10.712545394897461, 4.604351043701172]"
31959,0,[],"['song', 'morning']","[-16.49297332763672, 33.97321319580078]","[-0.5365710854530334, -12.591863632202148]"
31960,1,"['sikh', 'temple', 'calgary', 'wso']","['sikh', 'vandalised', 'wso', 'act']","[5.2635626792907715, 43.18000030517578]","[22.43916130065918, -8.663960456848145]"


Split vector into columns

In [65]:
X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
X[["bow_hashtags_1", "bow_hashtags_2"]] = pd.DataFrame(X.bow_hashtags.tolist(), index= X.index)
X

/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[["bow_tokens_1", "bow_tokens_"]] = pd.DataFrame(X.bow_tokens.tolist(), index= X.index)
/var/folders/0q/8f0zskws27x_14w3qmfndjfw0000gn/T/ipykernel_14199/2517448426.py

,n_mentions,lemmatized_hashtags,lemmatized_tokens,bow_tokens,bow_hashtags,bow_tokens_1,bow_tokens_,bow_hashtags_1,bow_hashtags_2
0,1,['run'],"['father', 'dysfunctional', 'selfish', 'drag',...","[4.522366523742676, -12.791789054870605]","[3.0793397426605225, 12.407670974731445]",4.522367,-12.791789,3.079340,12.407671
1,2,"['lyft', 'disapointed', 'getthanked']","['user', 'credit', 'cant', 'cause', 'dont', 'w...","[-16.068601608276367, -44.1268196105957]","[0.5125786662101746, 1.6410552263259888]",-16.068602,-44.126820,0.512579,1.641055
2,0,[],[],"[-58.391571044921875, 23.286211013793945]","[-0.5365710854530334, -12.591863632202148]",-58.391571,23.286211,-0.536571,-12.591864
3,0,['model'],"['u', 'time', 'urð\x9f\x93±', 'ð\x9f\x92¦ð\x9f...","[58.901100158691406, 30.188573837280273]","[18.094478607177734, -19.416061401367188]",58.901100,30.188574,18.094479,-19.416061
4,0,['motivation'],"['society', 'motivation']","[-41.57936477661133, 31.845523834228516]","[7.747095108032227, -33.71648406982422]",-41.579365,31.845524,7.747095,-33.716484
...,...,...,...,...,...,...,...,...,...
31957,1,[],['user'],"[-20.37633514404297, 61.130393981933594]","[-0.5365710854530334, -12.591863632202148]",-20.376335,61.130394,-0.536571,-12.591864
31958,0,"['shame', 'imwithher']","['see', 'turner', 'trying', 'wrap', 'mantle', ...","[3.5070629119873047, 20.457685470581055]","[10.712545394897461, 4.604351043701172]",3.507063,20.457685,10.712545,4.604351
31959,0,[],"['song', 'morning']","[-16.49297332763672, 33.97321319580078]","[-0.5365710854530334, -12.591863632202148]",-16.492973,33.973213,-0.536571,-12.591864
31960,1,"['sikh', 'temple', 'calgary', 'wso']","['sikh', 'vandalised', 'wso', 'act']","[5.2635626792907715, 43.18000030517578]","[22.43916130065918, -8.663960456848145]",5.263563,43.180000,22.439161,-8.663960


## Splitting data


In [66]:
Data = X[["n_mentions", "bow_tokens_1", "bow_tokens_", "bow_hashtags_1", "bow_hashtags_2"]]
Data

,n_mentions,bow_tokens_1,bow_tokens_,bow_hashtags_1,bow_hashtags_2
0,1,4.522367,-12.791789,3.079340,12.407671
1,2,-16.068602,-44.126820,0.512579,1.641055
2,0,-58.391571,23.286211,-0.536571,-12.591864
3,0,58.901100,30.188574,18.094479,-19.416061
4,0,-41.579365,31.845524,7.747095,-33.716484
...,...,...,...,...,...
31957,1,-20.376335,61.130394,-0.536571,-12.591864
31958,0,3.507063,20.457685,10.712545,4.604351
31959,0,-16.492973,33.973213,-0.536571,-12.591864
31960,1,5.263563,43.180000,22.439161,-8.663960


In [67]:
X_train, X_test, y_train, y_test = train_test_split(Data, y, test_size=0.3, random_state=55)

len(X_train)

22373

## Fitting model

In [71]:
results = []
for i in range(1, 50):
    classifier = DecisionTreeClassifier(random_state=55, max_depth=i)

    model = classifier.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    #print("Accuracy:", accuracy_score(y_test, y_pred))
    #print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    results.append([i,accuracy_score(y_test, y_pred), f1_score(y_test, y_pred)])

print(results)
print(y_test.value_counts()[0]/len(y_test))

#plot_tree(classifier, max_depth=2)

[[1, 0.9278339764313276, 0.0], [2, 0.9278339764313276, 0.0], [3, 0.92950255501095, 0.08152173913043478], [4, 0.9306497027844405, 0.13298565840938725], [5, 0.9319011367191574, 0.12113055181695827], [6, 0.932839712170195, 0.1503957783641161], [7, 0.9335697152987799, 0.18437900128040974], [8, 0.9334654291375535, 0.18622448979591835], [9, 0.9337782876212327, 0.22085889570552147], [10, 0.9339868599436855, 0.2233128834355828], [11, 0.9334654291375535, 0.23132530120481926], [12, 0.9319011367191574, 0.24856156501726126], [13, 0.932318281364063, 0.26166097838452784], [14, 0.9314839920742517, 0.2708102108768035], [15, 0.9296068411721764, 0.2749731471535983], [16, 0.9286682657211388, 0.2860125260960334], [17, 0.9266868286578371, 0.2819203268641471], [18, 0.9249139639169882, 0.2828685258964143], [19, 0.9252268224006674, 0.2949852507374631], [20, 0.9231410991761393, 0.28792270531400965], [21, 0.9230368130149129, 0.29714285714285715], [22, 0.9182396495984982, 0.2885662431941924], [23, 0.916153926373

[[1, 0.9278339764313276], [2, 0.9278339764313276], [3, 0.9296068411721764], [4, 0.9297111273334029], [5, 0.9311711335905726], [6, 0.9316925643967046], [7, 0.9320054228803838], [8, 0.9322139952028365], [9, 0.931275419751799], [10, 0.9327354260089686], [11, 0.932318281364063], [12, 0.9316925643967046], [13, 0.9298154134946293], [14, 0.9300239858170821], [15, 0.9292939826884973], [16, 0.9285639795599124], [17, 0.9292939826884973], [18, 0.9271039733027427], [19, 0.9248096777557618], [20, 0.9244968192720826], [21, 0.9206382313067056], [22, 0.9203253728230264], [23, 0.9161539263739702], [24, 0.9144853477943476], [25, 0.9147982062780269], [26, 0.911982479924914], [27, 0.9087496089268954], [28, 0.9072896026697257], [29, 0.9055167379288769], [30, 0.9042653039941599], [31, 0.9036395870268016], [32, 0.9040567316717072], [33, 0.9028052977369903], [34, 0.9028052977369903], [35, 0.903222442381896], [36, 0.9017624361247263], [37, 0.900928146834915], [38, 0.900928146834915], [39, 0.9011367191573678], [40, 0.9008238606736886], [41, 0.8987381374491605], [42, 0.8992595682552925], [43, 0.8989467097716133], [44, 0.8973824173532172], [45, 0.8958181249348212], [46, 0.8966524142246324], [47, 0.8961309834185004], [48, 0.8964438419021796], [49, 0.8957138387735948]]

Dataset split: 0.9278339764313276